In [9]:
import random
import ipywidgets as widgets
from IPython.display import display
import time
import asyncio
import threading

class MatchmakerGame:

    def __init__(self, rows, cols, time_limit=60):
        self.rows = rows
        self.cols = cols
        self.time_limit = time_limit
        self.start_time = time.time()

        self.buttons = {}
        self.button_symbols = {}
        self.revealed = []
        self.matched = set()

        self.timer_label = widgets.Label()
        self.status_label = widgets.Label()
        self.game_over = False

        total_cells = rows * cols
        symbol_count = total_cells // 2
        self.symbols = random.sample([chr(x) for x in range(65, 91)], symbol_count) * 2
        random.shuffle(self.symbols)

        self.create_widgets()
        self.start_timer()

    def create_widgets(self):
        self.grid_layout = widgets.GridspecLayout(self.rows, self.cols)

        for x in range(self.rows):
            for y in range(self.cols):
                button = widgets.Button(
                    description=' ',
                    layout=widgets.Layout(width='60px', height='50px')
                )
                button.on_click(lambda btn, x=x, y=y: asyncio.create_task(self.reveal_card(x, y)))
                self.buttons[x, y] = button
                self.button_symbols[x, y] = self.symbols.pop()
                self.grid_layout[x, y] = button

        display(widgets.VBox([self.timer_label, self.status_label, self.grid_layout]))

    async def reveal_card(self, x, y):
        if self.game_over or (x, y) in self.matched or (x, y) in self.revealed:
            return

        if len(self.revealed) == 2:
            return

        self.buttons[x, y].description = self.button_symbols[x, y]
        self.revealed.append((x, y))

        if len(self.revealed) == 2:
            await asyncio.sleep(0.5)
            await self.check_match()

    async def check_match(self):
        (x1, y1), (x2, y2) = self.revealed
        sym1 = self.button_symbols[x1, y1]
        sym2 = self.button_symbols[x2, y2]

        if sym1 == sym2:
            self.matched.update([(x1, y1), (x2, y2)])
        else:
            await asyncio.sleep(0.5)
            self.buttons[x1, y1].description = ' '
            self.buttons[x2, y2].description = ' '

        self.revealed.clear()

        if len(self.matched) == self.rows * self.cols:
            self.end_game(won=True)

    def start_timer(self):
        def update():
            while not self.game_over:
                remaining = int(self.time_limit - (time.time() - self.start_time))
                if remaining <= 0:
                    self.end_game(won=False)
                    break
                self.timer_label.value = f"⏳ Time Left: {remaining} seconds"
                time.sleep(1)

        threading.Thread(target=update, daemon=True).start()

    def end_game(self, won):
        self.game_over = True
        for button in self.buttons.values():
            button.disabled = True
        self.status_label.value = "🎉 You Won!" if won else "😢 You Lost! Time's up."

# 🎮 Start the game with 1-minute timer
game = MatchmakerGame(4, 6, time_limit=60)
